In [1]:
!pip install autots
# Read in data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import re
import time
import datetime
from sklearn.model_selection import train_test_split
from autots import AutoTS

In [2]:
import warnings

warnings.filterwarnings("ignore")

# Import data

In [3]:
#df

In [4]:
# Define years and months to load data from both 2023 and 2024
years = [2023, 2024]
months = range(1, 13)

# Load datasets dynamically for both years
data_files = [
    f"ig_data-{year}/ig_data_{str(month).zfill(2)}-{year}_u.csv"
    for year in years
    for month in months
]

datasets = [pd.read_csv(file) for file in data_files]

# Add month and year columns
for i, (year, month) in enumerate([(y, m) for y in years for m in months]):
    datasets[i]["month"] = month
    datasets[i]["year"] = year

In [5]:
# Concatenate data
data = pd.concat(datasets)
data

,date,profile_id,followers,posts,engagement,likes,comments,reach,impressions,month,year
0,2023-01-01,26dbe87d9f80099370c8e724c31eb0f3731afc2aff5f62...,20558,0,0,0,0,0,0,1,2023
1,2023-01-01,31832168975075c19bfca97103d24b2525b75235b2e61a...,5256,0,0,0,0,0,0,1,2023
2,2023-01-01,065c756ab25ca5147325477d859c320577aa171e55d99b...,337483,1,149,145,4,4101,4511,1,2023
3,2023-01-01,ed8304331e1ba4cb7828e07b5f768f67b622a50ea5e25a...,62556,0,0,0,0,0,0,1,2023
4,2023-01-01,27178b23de82c1fd075ab6e928dec79ea604d5afb2e4c4...,359455,1,3997,3509,488,57388,63127,1,2023
...,...,...,...,...,...,...,...,...,...,...,...
547988,2024-12-31,8c1ac76cb45374c9710165d154d153ddf48bc30c253c8e...,5905,0,0,0,0,0,0,12,2024
547989,2024-12-31,834715aec99b1e9be4feccadeea4712f1cde42d649664d...,211245,1,345,337,8,10157,11172,12,2024
547990,2024-12-31,94528532ec6bee015266fdaeb9cd570a6fbf6e3a2dfbc2...,1660,0,0,0,0,0,0,12,2024
547991,2024-12-31,0597c04c318d668379c1d549c31931c56838f6865ae7f3...,10084,1,28,28,0,2914,3205,12,2024


In [6]:
# find in the column followers the rows that have the value as "undefined"
data[data["followers"] == "undefined"]
# replace the "undefined" in follwoers with nan
data["followers_num"] = data["followers"].replace("undefined", np.nan)
# convert the followers_num to integer
data["followers_num"] = data["followers_num"].astype(float).astype(pd.Int64Dtype())
# Ensure followers_num is numeric and clean
data["followers_num"] = pd.to_numeric(data["followers_num"], errors="coerce")

# Step 4: Replace zeros with NaN (to avoid filling with zero)
data["followers_num"].replace(0, np.nan, inplace=True)

# Step 5: Fill NaN values using the last non-null value, making sure it isn't zero
data["followers_num"] = data["followers_num"].fillna(method="ffill")

# Step 6: If still NaN (if there were no valid previous non-null value), you can set to zero or some other value
data["followers_num"].fillna(0, inplace=True)

# Step 7: Ensure the column type is integer, but with nullable integer type to handle missing values
data["followers_num"] = data["followers_num"].astype(pd.Int64Dtype())

# Verify the changes
print(data.head())


         date                                         profile_id followers  \
0  2023-01-01  26dbe87d9f80099370c8e724c31eb0f3731afc2aff5f62...     20558   
1  2023-01-01  31832168975075c19bfca97103d24b2525b75235b2e61a...      5256   
2  2023-01-01  065c756ab25ca5147325477d859c320577aa171e55d99b...    337483   
3  2023-01-01  ed8304331e1ba4cb7828e07b5f768f67b622a50ea5e25a...     62556   
4  2023-01-01  27178b23de82c1fd075ab6e928dec79ea604d5afb2e4c4...    359455   

   posts  engagement  likes  comments  reach  impressions  month  year  \
0      0           0      0         0      0            0      1  2023   
1      0           0      0         0      0            0      1  2023   
2      1         149    145         4   4101         4511      1  2023   
3      0           0      0         0      0            0      1  2023   
4      1        3997   3509       488  57388        63127      1  2023   

   followers_num  
0          20558  
1           5256  
2         337483  
3         

In [7]:
data.isnull().sum()

date             0
profile_id       0
followers        0
posts            0
engagement       0
likes            0
comments         0
reach            0
impressions      0
month            0
year             0
followers_num    0
dtype: int64

In [8]:
data[data["followers_num"] == "undefined"]

,date,profile_id,followers,posts,engagement,likes,comments,reach,impressions,month,year,followers_num


In [9]:
profile_counts = data["profile_id"].value_counts()
# # get the ids of the accounts that have more than 365 records
# more_than_299_ids = profile_counts[profile_counts >= 300].index
# # filter the data to include only the accounts that have more than 365 records
# more_than_299_data = data[data["profile_id"].isin(more_than_299_ids)]
# get the ids of the accounts that have more than 365 records
more_than_365_ids = profile_counts[profile_counts > 365].index
# filter the data to include only the accounts that have more than 365 records
more_than_365_data = data[data["profile_id"].isin(more_than_365_ids)]
# get the ids of the accounts that have 300 to 365 records
between_365_300_ids = profile_counts[
    (profile_counts <= 365) & (profile_counts >= 300)
].index
# filter the data to include only the accounts that have 300 to 365 records
between_365_300_data = data[data["profile_id"].isin(between_365_300_ids)]
# get the ids of the accounts that have 90 to 299 records
between_299_90_ids = profile_counts[
    (profile_counts < 300) & (profile_counts >= 90)
].index
# filter the data to include only the accounts that have 90 to 299 records
between_299_90_data = data[data["profile_id"].isin(between_299_90_ids)]

In [12]:
# Export more than 365 data to csv
between_365_300_data.to_csv("between_365_300_data.csv", index=False)

# Export more than 365 data to csv
between_299_90_data.to_csv("between_299_90_data.csv", index=False)

# Monthly

## 365+

In [46]:
# Convert date column to datetime
more_than_365_data["date"] = pd.to_datetime(more_than_365_data["date"])

# Extract year and month
more_than_365_data["year"] = more_than_365_data["date"].dt.year
more_than_365_data["month"] = more_than_365_data["date"].dt.month

# Aggregate: Keep last follower count per month for each profile
monthly_df = (
    more_than_365_data.sort_values(["profile_id", "date"])
    .groupby(["profile_id", "year", "month"])
    .last()
    .reset_index()
)

# Convert back to date format (e.g., use the first day of the month)
monthly_df["date"] = pd.to_datetime(monthly_df[["year", "month"]].assign(day=1))

# Define the full date range (monthly from Jan 2023 to Dec 2024)
full_date_range = pd.date_range(start="2023-01-01", end="2024-12-01", freq="MS")  # 'MS' = Month Start

# Drop unnecessary columns
monthly_df = monthly_df[["date", "profile_id", "followers_num"]]

# Set a random seed for reproducibility
np.random.seed(42)  # You can use any number as the seed

# Get unique profile IDs
unique_ids = monthly_df["profile_id"].unique()

# Create a MultiIndex with all profiles and all months
full_index = pd.MultiIndex.from_product([unique_ids, full_date_range], names=["profile_id", "date"])

# Reindex the DataFrame to enforce this structure
monthly_df = monthly_df.set_index(["profile_id", "date"]).reindex(full_index).reset_index()

# Randomly select 60 IDs
selected_ids = np.random.choice(unique_ids, size=60, replace=False)

# Filter the DataFrame to include only the selected IDs
monthly_df = monthly_df[monthly_df['profile_id'].isin(selected_ids)]

# Verify the number of unique IDs in the filtered DataFrame
print(f"Number of unique IDs in filtered DataFrame: {monthly_df['profile_id'].nunique()}")

Number of unique IDs in filtered DataFrame: 60


#### train-test split 75%

In [47]:
monthly_df.fillna(method='ffill', inplace=True)
monthly_df.fillna(method='bfill', inplace=True)
monthly_df.fillna(0, inplace=True)
# Split the data 
train_df = monthly_df[monthly_df['date'] < '2024-07-01']
test_df = monthly_df[monthly_df['date'] >= '2024-07-01']
# Pivot and convert dates to month-end format
def prepare_for_autts(df):
    df = df.pivot(index='date', columns='profile_id', values='followers_num')
    df = df.fillna(method='ffill')
    df.index = df.index.to_period('M').to_timestamp('M')
    df = df.fillna(method='ffill')
    return df

train_df = prepare_for_autts(train_df)
test_df = prepare_for_autts(test_df)
# Ensure no missing values
train_df.fillna(method='ffill', inplace=True)
train_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)
# Ensure no missing values
test_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='bfill', inplace=True)
test_df.fillna(0, inplace=True)

In [48]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Store evaluation results
evaluation_results = []

# ----- AUTOTS MODEL -----
from autots import AutoTS

# Initialize and train the AutoTS model (without 'fillna' argument)
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='M',
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate AutoTS model
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    mae_autots = mean_absolute_error(true_values, predicted_values)
    mse_autots = mean_squared_error(true_values, predicted_values)
    evaluation_results.append({'Profile': profile_id, 'MAE_AutoTS_365': mae_autots, 'MSE_AutoTS_365': mse_autots})

# ----- CREATE COMPARISON DATAFRAME -----
evaluation_df = pd.DataFrame(evaluation_results)


Using 4 cpus for n_jobs.
Data frequency is: M, used frequency is: M
Too many training validations for length of data provided, decreasing num_validations to 1
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10
Model Number: 6 with model DatepartRegression in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/

Model Number: 7 with model DatepartRegression in generation 0 of 10
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7 in generation 0: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 10
Model Number: 9 with model ETS in generation 0 of 10
Model Number: 10 with model GLM in generation 0 of 10
Model Number: 11 with model GLM in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or p

Model Number: 12 with model GLS in generation 0 of 10
Model Number: 13 with model GLS in generation 0 of 10
Model Number: 14 with model LastValueNaive in generation 0 of 10
Model Number: 15 with model LastValueNaive in generation 0 of 10
Model Number: 16 with model LastValueNaive in generation 0 of 10
Model Number: 17 with model LastValueNaive in generation 0 of 10
Model Number: 18 with model SeasonalNaive in generation 0 of 10
Model Number: 19 with model SeasonalNaive in generation 0 of 10
Model Number: 20 with model SeasonalNaive in generation 0 of 10
Model Number: 21 with model VAR in generation 0 of 10
Template Eval Error: ValueError('maxlags is too large for the number of observations and the number of equations. The largest model cannot be estimated.') in model 21 in generation 0: VAR
Model Number: 22 with model VAR in generation 0 of 10
Template Eval Error: ValueError('maxlags is too large for the number of observations and the number of equations. The largest model cannot be es

...

Model Number: 955 with model RRVAR in generation 9 of 10
Model Number: 956 with model LastValueNaive in generation 9 of 10
Model Number: 957 with model LastValueNaive in generation 9 of 10
Model Number: 958 with model DatepartRegression in generation 9 of 10
Model Number: 959 with model AverageValueNaive in generation 9 of 10
Model Number: 960 with model ConstantNaive in generation 9 of 10
Model Number: 961 with model ConstantNaive in generation 9 of 10
Model Number: 962 with model GLS in generation 9 of 10
Model Number: 963 with model SeasonalityMotif in generation 9 of 10
Model Number: 964 with model BasicLinearModel in generation 9 of 10
Model Number: 965 with model LastValueNaive in generation 9 of 10
Template Eval Error: Exception("Transformer AlignLastValue failed on fit from params ffill_mean_biased {'0': {'rows': 1, 'lag': 84, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 3, 'threshold_method': 'max'}, '1': {'lag_1': 7, 'method': 2}, '2': {}, '3

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1650: RuntimeWarning: invalid value encountered in log
  endog * np.log(endog / mu) + (mu - endog))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1650: RuntimeWarning: invalid value encountered in log
  endog * np.log(endog / mu) + (mu - endog))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1650: RuntimeWarning: invalid value encountered in log
  endog * np.log(endog / mu) + (mu - endog))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1650: RuntimeWarning: invalid value encountered in log
  endog * np.log(endog / mu) + (mu - endog))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1650: RuntimeWarning: divide by zero encountered in divide
  endog * np.log

Model Number: 973 with model LastValueNaive in generation 9 of 10
Model Number: 974 with model SeasonalityMotif in generation 9 of 10
Template Eval Error: ValueError('window shape cannot be larger than input array shape') in model 974 in generation 9: SeasonalityMotif
Model Number: 975 with model SeasonalNaive in generation 9 of 10
Model Number: 976 with model GLS in generation 9 of 10
Model Number: 977 with model FFT in generation 9 of 10
Model Number: 978 with model BasicLinearModel in generation 9 of 10
Template Eval Error: ValueError("Model returned NaN due to a preprocessing transformer {'fillna': 'zero', 'transformations': {'0': 'Slice', '1': 'MaxAbsScaler', '2': 'AlignLastValue', '3': 'RobustScaler', '4': 'DifferencedTransformer'}, 'transformation_params': {'0': {'method': 100}, '1': {}, '2': {'rows': 168, 'lag': 28, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 1, 'threshold_method': 'max'}, '3': {}, '4': {'lag': 1, 'fill': 'zero'}}}. fail_on_fo

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:445: RuntimeWarning: divide by zero encountered in divide
  endog_mu = self._clean(endog / mu)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:143: RuntimeWarning: divide by zero encountered in divide
  return 1. / (self.link.deriv(mu)**2 * self.variance(mu))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:445: RuntimeWarning: divide by zero encountered in divide
  endog_mu = self._clean(endog / mu)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:143: RuntimeWarning: divide by zero encountered in divide
  return 1. / (self.link.deriv(mu)**2 * self.variance(mu))
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:445: RuntimeWarning: divide by zero encountered in 

Model Number: 1017 with model ConstantNaive in generation 10 of 10
Model Number: 1018 with model SeasonalityMotif in generation 10 of 10
Model Number: 1019 with model AverageValueNaive in generation 10 of 10
Model Number: 1020 with model LastValueNaive in generation 10 of 10
Model Number: 1021 with model RRVAR in generation 10 of 10
Model Number: 1022 with model AverageValueNaive in generation 10 of 10
Model Number: 1023 with model ConstantNaive in generation 10 of 10
Model Number: 1024 with model ETS in generation 10 of 10
Model Number: 1025 with model ETS in generation 10 of 10
Model Number: 1026 with model ConstantNaive in generation 10 of 10
Model Number: 1027 with model BasicLinearModel in generation 10 of 10
Model Number: 1028 with model AverageValueNaive in generation 10 of 10
Model Number: 1029 with model SeasonalNaive in generation 10 of 10
Model Number: 1030 with model ETS in generation 10 of 10
Model Number: 1031 with model SeasonalNaive in generation 10 of 10
Model Number: 

In [53]:
evaluation_results

[{'Profile': '02170cc6ad8d4839d4b103387977def165bea9631fd28de5379a59339bc6b942',
  'MAE_AutoTS_365': 1221.0,
  'MSE_AutoTS_365': 1490841.0},
 {'Profile': '0225ee6b651c06a7b02cf995cb466c63dc02b861b04fc5294457e5cd17c67267',
  'MAE_AutoTS_365': 17930.0,
  'MSE_AutoTS_365': 321484900.0},
 {'Profile': '077e1d3b46ac249ee51fdb604c82dfbdf350f46a48fe2ca20ea3bc11614cec64',
  'MAE_AutoTS_365': 187200.0,
  'MSE_AutoTS_365': 35043840000.0},
 {'Profile': '0c997f518f0769c39126acb6538cae0f9c945346abecc99705370a49c377467e',
  'MAE_AutoTS_365': 646770.0,
  'MSE_AutoTS_365': 418311432900.0},
 {'Profile': '100b80f30629b949b2b3015e944f81dbdee219520fecab758468d3e0c433e8f1',
  'MAE_AutoTS_365': 904.5,
  'MSE_AutoTS_365': 818125.1666666666},
 {'Profile': '10bcfb1593e1c348b04c1aa2aeb4d1add22a9ffe71ba7340e9649574b01b1dbe',
  'MAE_AutoTS_365': 6816.0,
  'MSE_AutoTS_365': 46457856.0},
 {'Profile': '13ee2c9dc0adbe540b53810f8dd772aabba4f8db2fa634bcdfa14b7b4822158a',
  'MAE_AutoTS_365': 49329.0,
  'MSE_AutoTS_365': 

In [34]:
import numpy as np
import pandas as pd

profile_ids = test_df.columns

# Initialize a dictionary to store results for each profile_id
results = {}

# Loop through each profile_id
for profile_id in profile_ids:
    # Step 1: Calculate the confidence interval for true follower numbers (followers_num)
    mean_followers = test_df_aligned[profile_id].mean()
    std_followers = test_df_aligned[profile_id].std()

    # Calculate the 95% confidence interval for true followers
    confidence_interval_lower = max(0, mean_followers - 1.96 * (std_followers / np.sqrt(len(test_df_aligned[profile_id]))))
    confidence_interval_upper = mean_followers + 1.96 * (std_followers / np.sqrt(len(test_df_aligned[profile_id])))

    # Step 2: Check how many predictions fall within the confidence interval
    predictions_within_interval = (
        (forecast_df[profile_id] >= confidence_interval_lower) & 
        (forecast_df[profile_id] <= confidence_interval_upper)
    )

    # Step 3: Calculate the number of predictions within the confidence interval
    num_predictions_within_interval = len(predictions_within_interval)

    # Step 4: Calculate the ratio of predictions within the confidence interval
    ratio_within_interval = num_predictions_within_interval / len(test_df_aligned[profile_id])

    # Store results for this profile_id
    results[profile_id] = {
        "confidence_interval_lower": confidence_interval_lower,
        "confidence_interval_upper": confidence_interval_upper,
        "num_predictions_within_interval": num_predictions_within_interval,
        "ratio_within_interval": ratio_within_interval,
    }

# Convert results to a DataFrame for better visualization
results_df = pd.DataFrame(results).T
results_df

,confidence_interval_lower,confidence_interval_upper,num_predictions_within_interval,ratio_within_interval
02170cc6ad8d4839d4b103387977def165bea9631fd28de5379a59339bc6b942,1221.000000,1221.000000,6.0,1.0
0225ee6b651c06a7b02cf995cb466c63dc02b861b04fc5294457e5cd17c67267,17930.000000,17930.000000,6.0,1.0
077e1d3b46ac249ee51fdb604c82dfbdf350f46a48fe2ca20ea3bc11614cec64,187200.000000,187200.000000,6.0,1.0
0c997f518f0769c39126acb6538cae0f9c945346abecc99705370a49c377467e,646770.000000,646770.000000,6.0,1.0
100b80f30629b949b2b3015e944f81dbdee219520fecab758468d3e0c433e8f1,902.556402,906.443598,6.0,1.0
10bcfb1593e1c348b04c1aa2aeb4d1add22a9ffe71ba7340e9649574b01b1dbe,6816.000000,6816.000000,6.0,1.0
13ee2c9dc0adbe540b53810f8dd772aabba4f8db2fa634bcdfa14b7b4822158a,49329.000000,49329.000000,6.0,1.0
226f63965190887c5bebae105d865de652d7141ce1927db9e10609a193d04901,2332.000000,2332.000000,6.0,1.0
241b583fb8fe8bb759ef5ad6159ac45a0c78470e7718ae5a864a59b46705431a,34440.324919,66984.675081,6.0,1.0
2ad2fbc6a393c64ed2acfaeb4ef79119a6c1235424dd79ff8872391041db20e6,425.000000,425.000000,6.0,1.0


## 300-365

In [54]:
# Convert date column to datetime
between_365_300_data["date"] = pd.to_datetime(between_365_300_data["date"])

# Extract year and month
between_365_300_data["year"] = between_365_300_data["date"].dt.year
between_365_300_data["month"] = between_365_300_data["date"].dt.month

# Aggregate: Keep last follower count per month for each profile
monthly_df = (
    between_365_300_data.sort_values(["profile_id", "date"])
    .groupby(["profile_id", "year", "month"])
    .last()
    .reset_index()
)

# Convert back to date format (e.g., use the first day of the month)
monthly_df["date"] = pd.to_datetime(monthly_df[["year", "month"]].assign(day=1))

# Define the full date range (monthly from Jan 2023 to Dec 2024)
full_date_range = pd.date_range(start="2023-01-01", end="2024-12-01", freq="MS")  # 'MS' = Month Start

# Drop unnecessary columns
monthly_df = monthly_df[["date", "profile_id", "followers_num"]]

# Set a random seed for reproducibility
np.random.seed(42)  # You can use any number as the seed

# Get unique profile IDs
unique_ids = monthly_df["profile_id"].unique()

# Create a MultiIndex with all profiles and all months
full_index = pd.MultiIndex.from_product([unique_ids, full_date_range], names=["profile_id", "date"])

# Reindex the DataFrame to enforce this structure
monthly_df = monthly_df.set_index(["profile_id", "date"]).reindex(full_index).reset_index()

# Randomly select 60 IDs
selected_ids = np.random.choice(unique_ids, size=60, replace=False)

# Filter the DataFrame to include only the selected IDs
monthly_df = monthly_df[monthly_df['profile_id'].isin(selected_ids)]

# Verify the number of unique IDs in the filtered DataFrame
#print(f"Number of unique IDs in filtered DataFrame: {monthly_df['profile_id'].nunique()}")

monthly_df.fillna(method='ffill', inplace=True)
monthly_df.fillna(method='bfill', inplace=True)
monthly_df.fillna(0, inplace=True)

# Split the data 
train_df = monthly_df[monthly_df['date'] < '2024-07-01']
test_df = monthly_df[monthly_df['date'] >= '2024-07-01']
# Pivot and convert dates to month-end format
def prepare_for_autts(df):
    df = df.pivot(index='date', columns='profile_id', values='followers_num')
    df = df.fillna(method='ffill')
    df.index = df.index.to_period('M').to_timestamp('M')
    df = df.fillna(method='ffill')
    return df

train_df = prepare_for_autts(train_df)
test_df = prepare_for_autts(test_df)
# Ensure no missing values
train_df.fillna(method='ffill', inplace=True)
train_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)
# Ensure no missing values
test_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='bfill', inplace=True)
test_df.fillna(0, inplace=True)

evaluation_results_300 = []

# Initialize and train the AutoTS model (without 'fillna' argument)
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='M',
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate AutoTS model
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    mae_autots = mean_absolute_error(true_values, predicted_values)
    mse_autots = mean_squared_error(true_values, predicted_values)
    evaluation_results_300.append({'Profile': profile_id, 'MAE_AutoTS_300': mae_autots, 'MSE_AutoTS_300': mse_autots})

# ----- CREATE COMPARISON DATAFRAME -----
evaluation_df_300 = pd.DataFrame(evaluation_results_300)

Using 4 cpus for n_jobs.
Data frequency is: M, used frequency is: M
Too many training validations for length of data provided, decreasing num_validations to 1
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py

Model Number: 6 with model DatepartRegression in generation 0 of 10
Model Number: 7 with model DatepartRegression in generation 0 of 10
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7 in generation 0: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 10
Model Number: 9 with model ETS in generation 0 of 10
Model Number: 10 with model GLM in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or p

Model Number: 11 with model GLM in generation 0 of 10
Model Number: 12 with model GLS in generation 0 of 10


...

31 - Ensemble with avg smape 114.52: 
32 - Ensemble with avg smape 114.52: 
33 - Ensemble with avg smape 0.0: 
34 - Ensemble with avg smape 133.76: 
Template Eval Error: Exception("Transformer SeasonalDifference failed on fit from params ffill {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 0.7, 'first_value_only': False, 'threshold': 1, 'threshold_method': 'max'}, '1': {'lag_1': 7, 'method': 20}, '2': {'rows': 1, 'lag': 28, 'method': 'multiplicative', 'strength': 1.0, 'first_value_only': False, 'threshold': None, 'threshold_method': 'mean'}} with error ValueError('cannot reshape array of size 360 into shape (0,7,newaxis)')") in model 34 in generation 0: Ensemble
Model Number: 35 of 16 with model Ensemble for Validation 1
35 - Ensemble with avg smape 114.94: 
36 - Ensemble with avg smape 114.94: 
37 - Ensemble with avg smape 0.0: 
38 - Ensemble with avg smape 133.62: 
Template Eval Error: Exception("Transformer SeasonalDifference failed on fit from params ffill {'0': {'ro

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


43 - Ensemble with avg smape 115.0: 
44 - Ensemble with avg smape 115.0: 
45 - Ensemble with avg smape 0.0: 
46 - Ensemble with avg smape 133.65: 
Template Eval Error: Exception("Transformer SeasonalDifference failed on fit from params ffill {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 0.7, 'first_value_only': False, 'threshold': 1, 'threshold_method': 'max'}, '1': {'lag_1': 7, 'method': 20}, '2': {'rows': 1, 'lag': 28, 'method': 'multiplicative', 'strength': 1.0, 'first_value_only': False, 'threshold': None, 'threshold_method': 'mean'}} with error ValueError('cannot reshape array of size 360 into shape (0,7,newaxis)')") in model 46 in generation 0: Ensemble
Model Number: 47 of 16 with model Ensemble for Validation 1
47 - Ensemble with avg smape 114.52: 
48 - Ensemble with avg smape 114.52: 
49 - Ensemble with avg smape 0.0: 
50 - Ensemble with avg smape 133.77: 
Template Eval Error: Exception("Transformer SeasonalDifference failed on fit from params ffill {'0': {'rows

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


55 - Ensemble with avg smape 114.45: 
56 - Ensemble with avg smape 114.45: 
57 - Ensemble with avg smape 0.0: 
58 - Ensemble with avg smape 133.77: 
Template Eval Error: Exception("Transformer SeasonalDifference failed on fit from params ffill {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 0.7, 'first_value_only': False, 'threshold': 1, 'threshold_method': 'max'}, '1': {'lag_1': 7, 'method': 20}, '2': {'rows': 1, 'lag': 28, 'method': 'multiplicative', 'strength': 1.0, 'first_value_only': False, 'threshold': None, 'threshold_method': 'mean'}} with error ValueError('cannot reshape array of size 360 into shape (0,7,newaxis)')") in model 58 in generation 0: Ensemble


In [55]:
evaluation_results_300

[{'Profile': '0385a1893cfc3d31caf2f904717039ed5bd40152626c32c9c735c7e8de0ea05c',
  'MAE_AutoTS_300': 19516.0,
  'MSE_AutoTS_300': 380874256.0},
 {'Profile': '03909ba84c6faf1d285deebf0b9be9d557d6b8f6fa358b9beb8fa8018bd9a4c0',
  'MAE_AutoTS_300': 1828001.0,
  'MSE_AutoTS_300': 3341587656001.0},
 {'Profile': '0a7eb9fd8ffc224e541cb120178dc83f1fd31d73b5cc082eba372f2bf948ef9d',
  'MAE_AutoTS_300': 20385.0,
  'MSE_AutoTS_300': 415548225.0},
 {'Profile': '1ee398aee66750f89e8ae811739ccb99734381991d2fc3f5fda10d0e573af499',
  'MAE_AutoTS_300': 378.0,
  'MSE_AutoTS_300': 142884.0},
 {'Profile': '1f64d05fc9bbce293c141cca321106fafa26076d0737707a3826e037116ad24d',
  'MAE_AutoTS_300': 4449.0,
  'MSE_AutoTS_300': 19793601.0},
 {'Profile': '23e9b065f30942d65992fc2290f02e57d8eb137ae88a6d3d11a0f6f9cf9771a4',
  'MAE_AutoTS_300': 153429.0,
  'MSE_AutoTS_300': 23540458041.0},
 {'Profile': '2daf9e6afde6b0ec7702156af97259f6a60a863ba55beb75b29cfef84efe74a8',
  'MAE_AutoTS_300': 1199.0,
  'MSE_AutoTS_300': 14379

## 50-299

In [57]:
# Convert date column to datetime
between_299_50_data["date"] = pd.to_datetime(between_299_50_data["date"])

# Extract year and month
between_299_50_data["year"] = between_299_50_data["date"].dt.year
between_299_50_data["month"] = between_299_50_data["date"].dt.month

# Aggregate: Keep last follower count per month for each profile
monthly_df = (
    between_299_50_data.sort_values(["profile_id", "date"])
    .groupby(["profile_id", "year", "month"])
    .last()
    .reset_index()
)

# Convert back to date format (e.g., use the first day of the month)
monthly_df["date"] = pd.to_datetime(monthly_df[["year", "month"]].assign(day=1))

# Define the full date range (monthly from Jan 2023 to Dec 2024)
full_date_range = pd.date_range(start="2023-01-01", end="2024-12-01", freq="MS")  # 'MS' = Month Start

# Drop unnecessary columns
monthly_df = monthly_df[["date", "profile_id", "followers_num"]]

# Set a random seed for reproducibility
np.random.seed(42)  # You can use any number as the seed

# Get unique profile IDs
unique_ids = monthly_df["profile_id"].unique()

# Create a MultiIndex with all profiles and all months
full_index = pd.MultiIndex.from_product([unique_ids, full_date_range], names=["profile_id", "date"])

# Reindex the DataFrame to enforce this structure
monthly_df = monthly_df.set_index(["profile_id", "date"]).reindex(full_index).reset_index()


# Randomly select 60 IDs
selected_ids = np.random.choice(unique_ids, size=60, replace=False)

# Filter the DataFrame to include only the selected IDs
monthly_df = monthly_df[monthly_df['profile_id'].isin(selected_ids)]

# Verify the number of unique IDs in the filtered DataFrame
#print(f"Number of unique IDs in filtered DataFrame: {monthly_df['profile_id'].nunique()}")

monthly_df.fillna(method='ffill', inplace=True)
monthly_df.fillna(method='bfill', inplace=True)
monthly_df.fillna(0, inplace=True)
# Split the data 
train_df = monthly_df[monthly_df['date'] < '2024-07-01']
test_df = monthly_df[monthly_df['date'] >= '2024-07-01']
# Pivot and convert dates to month-end format
def prepare_for_autts(df):
    df = df.pivot(index='date', columns='profile_id', values='followers_num')
    df = df.fillna(method='ffill')
    df.index = df.index.to_period('M').to_timestamp('M')
    df = df.fillna(method='ffill')
    return df

train_df = prepare_for_autts(train_df)
test_df = prepare_for_autts(test_df)
# Ensure no missing values
train_df.fillna(method='ffill', inplace=True)
train_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)
# Ensure no missing values
test_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='bfill', inplace=True)
test_df.fillna(0, inplace=True)

evaluation_results_50 = []

# Initialize and train the AutoTS model (without 'fillna' argument)
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='M',
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate AutoTS model
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    mae_autots = mean_absolute_error(true_values, predicted_values)
    mse_autots = mean_squared_error(true_values, predicted_values)
    evaluation_results_50.append({'Profile': profile_id, 'MAE_AutoTS_50': mae_autots, 'MSE_AutoTS_50': mse_autots})

# ----- CREATE COMPARISON DATAFRAME -----
evaluation_df_50 = pd.DataFrame(evaluation_results_50)

Using 4 cpus for n_jobs.
Data frequency is: M, used frequency is: M
Too many training validations for length of data provided, decreasing num_validations to 1
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py

Model Number: 6 with model DatepartRegression in generation 0 of 10
Model Number: 7 with model DatepartRegression in generation 0 of 10
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7 in generation 0: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 10
Model Number: 9 with model ETS in generation 0 of 10
Model Number: 10 with model GLM in generation 0 of 10
Model Number: 11 with model GLM in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or p

Model Number: 12 with model GLS in generation 0 of 10
Model Number: 13 with model GLS in generation 0 of 10
Model Number: 14 with model LastValueNaive in generation 0 of 10
Model Number: 15 with model LastValueNaive in generation 0 of 10
Model Number: 16 with model LastValueNaive in generation 0 of 10
Model Number: 17 with model LastValueNaive in generation 0 of 10
Model Number: 18 with model SeasonalNaive in generation 0 of 10
Template Eval Error: Exception("Transformer PowerTransformer failed on fit from params rolling_mean_24 {'0': {}, '1': {'model': 'middle', 'decimals': 2, 'on_transform': False, 'on_inverse': True}, '2': {}} with error BracketError('The algorithm terminated without finding a valid bracket. Consider trying different initial points.')") in model 18 in generation 0: SeasonalNaive
Model Number: 19 with model SeasonalNaive in generation 0 of 10
Model Number: 20 with model SeasonalNaive in generation 0 of 10
Model Number: 21 with model VAR in generation 0 of 10
Template

...

37 - Ensemble with avg smape 19.18: 
38 - Ensemble with avg smape 19.18: 
39 - Ensemble with avg smape 0.0: 
40 - Ensemble with avg smape 37.59: 
Template Eval Error: Exception("Transformer SeasonalDifference failed on fit from params ffill {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 0.7, 'first_value_only': False, 'threshold': 1, 'threshold_method': 'max'}, '1': {'lag_1': 7, 'method': 20}, '2': {'rows': 1, 'lag': 28, 'method': 'multiplicative', 'strength': 1.0, 'first_value_only': False, 'threshold': None, 'threshold_method': 'mean'}} with error ValueError('cannot reshape array of size 360 into shape (0,7,newaxis)')") in model 40 in generation 0: Ensemble
Model Number: 41 of 16 with model Ensemble for Validation 1


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


41 - Ensemble with avg smape 42.7: 
42 - Ensemble with avg smape 38.94: 
43 - Ensemble with avg smape 0.0: 
44 - Ensemble with avg smape 43.33: 
Template Eval Error: Exception("Transformer SeasonalDifference failed on fit from params ffill {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 0.7, 'first_value_only': False, 'threshold': 1, 'threshold_method': 'max'}, '1': {'lag_1': 7, 'method': 20}, '2': {'rows': 1, 'lag': 28, 'method': 'multiplicative', 'strength': 1.0, 'first_value_only': False, 'threshold': None, 'threshold_method': 'mean'}} with error ValueError('cannot reshape array of size 360 into shape (0,7,newaxis)')") in model 44 in generation 0: Ensemble
Model Number: 45 of 16 with model Ensemble for Validation 1
45 - Ensemble with avg smape 39.61: 
46 - Ensemble with avg smape 36.69: 
47 - Ensemble with avg smape 0.0: 
48 - Ensemble with avg smape 42.64: 
Template Eval Error: Exception("Transformer SeasonalDifference failed on fit from params ffill {'0': {'rows': 1,

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)


prediction too long for indepedent=False, falling back on indepedent=True
57 - Ensemble with avg smape 17.85: 
58 - Ensemble with avg smape 17.85: 
59 - Ensemble with avg smape 0.0: 
60 - Ensemble with avg smape 37.71: 
Template Eval Error: Exception("Transformer SeasonalDifference failed on fit from params ffill {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 0.7, 'first_value_only': False, 'threshold': 1, 'threshold_method': 'max'}, '1': {'lag_1': 7, 'method': 20}, '2': {'rows': 1, 'lag': 28, 'method': 'multiplicative', 'strength': 1.0, 'first_value_only': False, 'threshold': None, 'threshold_method': 'mean'}} with error ValueError('cannot reshape array of size 360 into shape (0,7,newaxis)')") in model 60 in generation 0: Ensemble
Model Number: 61 of 16 with model Ensemble for Validation 1
prediction too long for indepedent=False, falling back on indepedent=True
61 - Ensemble with avg smape 33.38: 
62 - Ensemble with avg smape 31.21: 
63 - Ensemble with avg smape 0.0: 


In [58]:
evaluation_results_50

[{'Profile': '02af12a05e0bc918bf3d11c82f0bdd9c8eb2d468023d4576c90e248c402d406d',
  'MAE_AutoTS_50': 0.0,
  'MSE_AutoTS_50': 0.0},
 {'Profile': '06653e0b982552df39bac305d78b9aa5610c08d08a51c8b10ca95871aa5de811',
  'MAE_AutoTS_50': 1338.5,
  'MSE_AutoTS_50': 3590005.8333333335},
 {'Profile': '07f06c4ea2484850d6b362de8264c8164df08dce850056eddd480e6152470c5a',
  'MAE_AutoTS_50': 0.0,
  'MSE_AutoTS_50': 0.0},
 {'Profile': '0a9f45c0c7b752bcef98c7d72e24461a39a89ebdcc6c5e67cb415208a0e388ea',
  'MAE_AutoTS_50': 30223.666666666668,
  'MSE_AutoTS_50': 1115421675.3333333},
 {'Profile': '10c7d077f54ccce4e526ffb0a0e182a546bfe4cad4a06140945d721cdb1bbb96',
  'MAE_AutoTS_50': 0.0,
  'MSE_AutoTS_50': 0.0},
 {'Profile': '12834d560e14aefe54636209b8552fc6fb50cb7c608d3cde5051031604d7612d',
  'MAE_AutoTS_50': 0.0,
  'MSE_AutoTS_50': 0.0},
 {'Profile': '1851922e33d7d9ec1bd672238c5cb93762523ee35c5e92077ec7f2dd7a65c8b9',
  'MAE_AutoTS_50': 0.0,
  'MSE_AutoTS_50': 0.0},
 {'Profile': '1bdebb0f96d1039af76952ec728c

## 100-299

In [ ]:
# get the ids of the accounts that have 50 to 299 records
between_299_90_ids = profile_counts[
    (profile_counts < 300) & (profile_counts >= 90)
].index
# filter the data to include only the accounts that have 50 to 299 records
between_299_90_data = data[data["profile_id"].isin(between_299_90_ids)]

In [78]:
# Convert date column to datetime
between_299_90_data["date"] = pd.to_datetime(between_299_90_data["date"])

# Extract year and month
between_299_90_data["year"] = between_299_90_data["date"].dt.year
between_299_90_data["month"] = between_299_90_data["date"].dt.month

# Aggregate: Keep last follower count per month for each profile
monthly_df = (
    between_299_90_data.sort_values(["profile_id", "date"])
    .groupby(["profile_id", "year", "month"])
    .last()
    .reset_index()
)

# Convert back to date format (e.g., use the first day of the month)
monthly_df["date"] = pd.to_datetime(monthly_df[["year", "month"]].assign(day=1))

# Define the full date range (monthly from Jan 2023 to Dec 2024)
full_date_range = pd.date_range(start="2023-01-01", end="2024-12-01", freq="MS")  # 'MS' = Month Start

# Drop unnecessary columns
monthly_df = monthly_df[["date", "profile_id", "followers_num"]]

# Set a random seed for reproducibility
np.random.seed(42)  # You can use any number as the seed

# Get unique profile IDs
unique_ids = monthly_df["profile_id"].unique()

# Create a MultiIndex with all profiles and all months
full_index = pd.MultiIndex.from_product([unique_ids, full_date_range], names=["profile_id", "date"])

# Reindex the DataFrame to enforce this structure
monthly_df = monthly_df.set_index(["profile_id", "date"]).reindex(full_index).reset_index()


# Randomly select 60 IDs
selected_ids = np.random.choice(unique_ids, size=60, replace=False)

# Filter the DataFrame to include only the selected IDs
monthly_df = monthly_df[monthly_df['profile_id'].isin(selected_ids)]

# Verify the number of unique IDs in the filtered DataFrame
#print(f"Number of unique IDs in filtered DataFrame: {monthly_df['profile_id'].nunique()}")

monthly_df.fillna(method='ffill', inplace=True)
monthly_df.fillna(method='bfill', inplace=True)
monthly_df.fillna(0, inplace=True)
# Split the data 
train_df = monthly_df[monthly_df['date'] < '2024-07-01']
test_df = monthly_df[monthly_df['date'] >= '2024-07-01']
# Pivot and convert dates to month-end format
def prepare_for_autts(df):
    df = df.pivot(index='date', columns='profile_id', values='followers_num')
    df = df.fillna(method='ffill')
    df.index = df.index.to_period('M').to_timestamp('M')
    df = df.fillna(method='ffill')
    return df

train_df = prepare_for_autts(train_df)
test_df = prepare_for_autts(test_df)
# Ensure no missing values
train_df.fillna(method='ffill', inplace=True)
train_df.fillna(method='bfill', inplace=True)
train_df.fillna(0, inplace=True)
# Ensure no missing values
test_df.fillna(method='ffill', inplace=True)
test_df.fillna(method='bfill', inplace=True)
test_df.fillna(0, inplace=True)

evaluation_results_90 = []

# Initialize and train the AutoTS model (without 'fillna' argument)
model = AutoTS(
    forecast_length=len(test_df),  
    frequency='M',
    ensemble='simple',
    max_generations=10,
    num_validations=3,
    validation_method='backwards'
)

# Train the model
model = model.fit(train_df)

# Generate predictions
predictions = model.predict(prediction_interval=0.95, fail_on_forecast_nan=False)
forecast_df = predictions.forecast.fillna(0)

# Align test data with predictions
test_df_aligned = test_df.loc[forecast_df.index]

# Evaluate AutoTS model
for profile_id in test_df_aligned.columns:
    true_values = test_df_aligned[profile_id]
    predicted_values = forecast_df[profile_id]
    mae_autots = mean_absolute_error(true_values, predicted_values)
    mse_autots = mean_squared_error(true_values, predicted_values)
    evaluation_results_90.append({'Profile': profile_id, 'MAE_AutoTS_50': mae_autots, 'MSE_AutoTS_50': mse_autots})

# ----- CREATE COMPARISON DATAFRAME -----
evaluation_df_90 = pd.DataFrame(evaluation_results_90)

Using 4 cpus for n_jobs.
Data frequency is: M, used frequency is: M
Too many training validations for length of data provided, decreasing num_validations to 1
Model Number: 1 with model AverageValueNaive in generation 0 of 10
Model Number: 2 with model AverageValueNaive in generation 0 of 10
Model Number: 3 with model AverageValueNaive in generation 0 of 10
Model Number: 4 with model DatepartRegression in generation 0 of 10
Model Number: 5 with model DatepartRegression in generation 0 of 10
Model Number: 6 with model DatepartRegression in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/sklearn/svm/_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/zeus/miniconda3/envs/

Model Number: 7 with model DatepartRegression in generation 0 of 10
Template Eval Error: ImportError('Tensorflow not available, install with pip install tensorflow.') in model 7 in generation 0: DatepartRegression
Model Number: 8 with model ETS in generation 0 of 10
Model Number: 9 with model ETS in generation 0 of 10
Model Number: 10 with model GLM in generation 0 of 10
Model Number: 11 with model GLM in generation 0 of 10


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or p

Model Number: 12 with model GLS in generation 0 of 10
Model Number: 13 with model GLS in generation 0 of 10
Model Number: 14 with model LastValueNaive in generation 0 of 10
Model Number: 15 with model LastValueNaive in generation 0 of 10
Model Number: 16 with model LastValueNaive in generation 0 of 10
Model Number: 17 with model LastValueNaive in generation 0 of 10
Model Number: 18 with model SeasonalNaive in generation 0 of 10
Template Eval Error: Exception("Transformer PowerTransformer failed on fit from params rolling_mean_24 {'0': {}, '1': {'model': 'middle', 'decimals': 2, 'on_transform': False, 'on_inverse': True}, '2': {}} with error BracketError('The algorithm terminated without finding a valid bracket. Consider trying different initial points.')") in model 18 in generation 0: SeasonalNaive
Model Number: 19 with model SeasonalNaive in generation 0 of 10
Model Number: 20 with model SeasonalNaive in generation 0 of 10
Model Number: 21 with model VAR in generation 0 of 10
Template

...

119 - GLM with avg smape 15.76: 
Model Number: 120 of 126 with model FFT for Validation 1
120 - FFT with avg smape 42.13: 
Model Number: 121 of 126 with model SeasonalNaive for Validation 1
121 - SeasonalNaive with avg smape 22.76: 
Model Number: 122 of 126 with model GLS for Validation 1
Template Eval Error: Exception("Transformer AlignLastValue failed on fit from params akima {'0': {'method': 0.5}, '1': {'method': 0.2}, '2': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 0.2, 'first_value_only': False, 'threshold': None, 'threshold_method': 'max'}} with error IndexError('single positional indexer is out-of-bounds')") in model 122 in generation 0: GLS
Model Number: 123 of 126 with model GLS for Validation 1
123 - GLS with avg smape 15.24: 
Model Number: 124 of 126 with model GLM for Validation 1
124 - GLM with avg smape 14.98: 
Model Number: 125 of 126 with model GLS for Validation 1
125 - GLS with avg smape 33.79: 
Model Number: 126 of 126 with model GLS for Validation 1


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/generalized_linear_model.py:1342: PerfectSeparationWarning: Perfect separation or p

126 - GLS with avg smape 33.79: 
Model Number: 127 of 126 with model GLM for Validation 1
127 - GLM with avg smape 44.58: 
Model Number: 128 of 126 with model SeasonalNaive for Validation 1
128 - SeasonalNaive with avg smape 27.35: 
Model Number: 129 of 126 with model SeasonalNaive for Validation 1
129 - SeasonalNaive with avg smape 36.87: 
Model Number: 130 of 126 with model SeasonalNaive for Validation 1
Template Eval Error: Exception("Transformer AlignLastValue failed on fit from params rolling_mean {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 10, 'threshold_method': 'mean'}, '1': {'cutoff': 0.6, 'reverse': False, 'on_transform': True, 'on_inverse': False}, '2': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 0.2, 'first_value_only': False, 'threshold': None, 'threshold_method': 'max'}, '3': {'rows': 1, 'lag': 7, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 1, 'threshold_method': 

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter alpha not set. Using default value alpha=1.0.
  warnings.warn("Negative binomial dispersion parameter alpha not "
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/statsmodels/genmod/families/family.py:1367: ValueWarning: Negative binomial dispersion parameter al

131 - SeasonalNaive with avg smape 31.2: 
Model Number: 132 of 126 with model SeasonalNaive for Validation 1
Template Eval Error: Exception("Transformer AlignLastValue failed on fit from params rolling_mean {'0': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 10, 'threshold_method': 'mean'}, '1': {'cutoff': 0.6, 'reverse': False, 'on_transform': True, 'on_inverse': False}, '2': {'rows': 1, 'lag': 1, 'method': 'additive', 'strength': 0.2, 'first_value_only': False, 'threshold': None, 'threshold_method': 'max'}, '3': {'rows': 1, 'lag': 7, 'method': 'additive', 'strength': 1.0, 'first_value_only': False, 'threshold': 1, 'threshold_method': 'mean'}} with error IndexError('single positional indexer is out-of-bounds')") in model 132 in generation 0: SeasonalNaive
Model Number: 133 of 126 with model FFT for Validation 1
133 - FFT with avg smape 43.48: 
Model Number: 134 of 126 with model DatepartRegression for Validation 1
134 - DatepartRe

In [82]:
evaluation_results_90

[{'Profile': '00ed31934cb3ec073edcc77b23fd93bada5ee0047ddd31b60ce3f9e182820c8f',
  'MAE_AutoTS_50': 1549.5,
  'MSE_AutoTS_50': 2481048.8333333335},
 {'Profile': '0967bebbf092787a3d3400ce1c1a012d88cbc1af75b7f7d2850baec542285ca8',
  'MAE_AutoTS_50': 6391.166666666667,
  'MSE_AutoTS_50': 122547336.16666667},
 {'Profile': '0d40472f6bca0211ad17770e9990344d16cf05b1f0fcd5b19fabdae86b1bc779',
  'MAE_AutoTS_50': 69341.0,
  'MSE_AutoTS_50': 4821406351.666667},
 {'Profile': '103a0b7815fdec7b7920309d62b5bfb1d36bed9b909c68a44976cc452adb13e8',
  'MAE_AutoTS_50': 17863.833333333332,
  'MSE_AutoTS_50': 478675266.1666667},
 {'Profile': '14c9357c4e4621e88d078764f978b2d0afed460175dc69d073959eff2c272373',
  'MAE_AutoTS_50': 0.0,
  'MSE_AutoTS_50': 0.0},
 {'Profile': '1672870e984cf7cbc3d10e576d66acf1b25b5a029b2282d2930b5517dc075b30',
  'MAE_AutoTS_50': 997500.1666666666,
  'MSE_AutoTS_50': 1474838537987.1667},
 {'Profile': '167d0013957e4cbf435c12d3fd63e3001392dc37d7b90fd2ba5935ab4d2369b8',
  'MAE_AutoTS_50

In [81]:
evaluation_df_90

,Profile,MAE_AutoTS_50,MSE_AutoTS_50
0,00ed31934cb3ec073edcc77b23fd93bada5ee0047ddd31...,1549.500000,2.481049e+06
1,0967bebbf092787a3d3400ce1c1a012d88cbc1af75b7f7...,6391.166667,1.225473e+08
2,0d40472f6bca0211ad17770e9990344d16cf05b1f0fcd5...,69341.000000,4.821406e+09
3,103a0b7815fdec7b7920309d62b5bfb1d36bed9b909c68...,17863.833333,4.786753e+08
4,14c9357c4e4621e88d078764f978b2d0afed460175dc69...,0.000000,0.000000e+00
5,1672870e984cf7cbc3d10e576d66acf1b25b5a029b2282...,997500.166667,1.474839e+12
6,167d0013957e4cbf435c12d3fd63e3001392dc37d7b90f...,0.000000,0.000000e+00
7,1aa839d0b39346e243f7e5499b1afcc020ebf829307dda...,0.000000,0.000000e+00
8,1ea02215d567605cc3ae010a3783a1b3c4a98e283fb335...,1799.333333,3.407005e+06
9,1ec8fb36c578c721e00d4143c6d2c2f66bbffb4d7fdb58...,0.000000,0.000000e+00
